# 基于 Chat Completions API 实现外部函数调用

2023年6月20日，OpenAI 官方在 Chat Completions API 原有的三种不同角色设定（System, Assistant, User）基础上，新增了 Function Calling 功能。

[详见OpenAI Blog](https://openai.com/blog/function-calling-and-other-api-updates)

## 概述

`functions` 是 Chat Completion API 中的可选参数，用于提供函数定义。其目的是使 GPT 模型能够生成符合所提供定义的函数参数。请注意，API不会实际执行任何函数调用。开发人员需要使用GPT 模型输出来执行函数调用。

主要请求参数说明：

- **`messages` (array, Required)**

  A list of messages comprising the conversation so far. [Example Python code](https://cookbook.openai.com/examples/how_to_format_inputs_to_chatgpt_models).

  - **`function_call` (object, Optional)**
    
    The name and arguments of a function that should be called, as generated by the model.

    - **`name` (string, Required)**
            
      The name of the function to call.
      
    - **`arguments` (string, Required)**
            
      The arguments to call the function with, as generated by the model in JSON format. Note that the model does not always generate valid JSON, and may hallucinate parameters not defined by your function schema. Validate the arguments in your code before calling your function.


- **`functions` (array, Optional)**

  A list of functions the model may generate JSON inputs for.

  - **`name` (string, Required)**
            
    The name of the function to be called. Must be a-z, A-Z, 0-9, or contain underscores and dashes, with a maximum length of 64.
      
  - **`description` (string, Optional)**
            
    A description of what the function does, used by the model to choose when and how to call the function.

  - **`parameters` (object, Required)**
    
    The parameters the functions accepts, described as a JSON Schema object. See the [guide](https://platform.openai.com/docs/guides/gpt/function-calling) for examples, and the [JSON Schema reference](https://json-schema.org/understanding-json-schema/) for documentation about the format.
    
- **`function_call` (string or undefined, Optional)**

  Controls how the model responds to function calls. 
    - `none` means the model does not call a function, and responds to the end-user. 
    - `auto` means the model can pick between an end-user or calling a function. 
    - Specifying a particular function via `{"name": "my_function"}` forces the model to call that function. 

  `none` is the default when no functions are present. `auto` is the default if functions are present.


如果提供了 `functions` 参数，默认情况下，GPT 模型将决定在何时适当地使用其中一个函数。

可以通过将 `function_call` 参数设置为 `{"name": "<insert-function-name>"}` 来强制 API 使用指定函数。

同时，也支持通过将 `function_call` 参数设置为 `"none"` 来强制API不使用任何函数。

如果使用了某个函数，则响应中的输出将包含 `"finish_reason": "function_call"`，以及一个具有该函数名称和生成的函数参数的 `function_call` 对象。

## 安装依赖包

In [1]:
%pip install scipy tenacity tiktoken termcolor openai requests

Note: you may need to restart the kernel to use updated packages.
